# QUAL Check - DevRate Data Generation Tool

----


Let's install required libraries for runnign the script.

In [1]:
!pip install gitpython

### Initialization of the Script

First we import the installed libraires.

In [2]:
from git import *
import subprocess
import os
import re
import io
import time
import datetime

In [3]:
os.system('sudo ntpdate -s time.nist.gov')
os.system('timedatectl set-ntp 1')

256

Next we need to define the helper functions.

#### Printing Functions

In [4]:
def print_red(str):
    print ('\033[91m{}\033[00m'.format(str))
def print_green(str):
    print ('\033[92m{}\033[00m'.format(str))
def print_yellow(str):
    print ('\033[93m{}\033[00m'.format(str))
def print_blue(str):
    print ('\033[94m{}\033[00m'.format(str))
def print_purple(str):
    print ('\033[95m{}\033[00m'.format(str))
def print_cyan(str):
    print("\033[96m {}\033[00m" .format(str))
def print_lightgray(str):
    print("\033[97m {}\033[00m" .format(str))
def print_black(str):
    print("\033[98m {}\033[00m" .format(str))

def print_format_table():
    for style in range(8):
        for foreground in range(30, 38):
            s1 = ''
            for background in range(40, 48):
                format = ';'.join([str(style), str(foreground), str(background)])
                s1 += '\x1b[%sm %s \x1b[0m' % (format, format)
            print(s1)
        print('\n')

### Retrieve GIT Commmits

Let's define configurations as variables.

#### Declare Variables

In [5]:
project_name = 'open-event-frontend'

Now we can retieve the commits list from repo.

#### Get Commits List

In [6]:
repo = Repo("repos/%s" % (project_name))
assert repo.bare == False

In [7]:
print(os.getcwd())

git = repo.git
logs_master = git.execute(["git", "log", "--first-parent", "development", "--after=\"2020-01-01\"", "--before=\"2021-01-01\"", "--pretty=format:%H", "--invert-grep", "--grep=chore(deps-dev)"])

commit_list = logs_master.split('\n')

print("Total Commits in the Git Repo: %d"%len(commit_list))

print(commit_list[0])

commit_list.reverse()

print(commit_list[0])

startat = 1
stopat = len(commit_list)

/home/jupyter
Total Commits in the Git Repo: 728
cfce6d291c553f9c33faf8a6307e3ec3c0d29fd6
25b1a767afe7d52d95b7757f902883d9bf5adfd1


#### Write commits SHAs to a file

In [8]:
repo_path = "repos/%s" % project_name
print('Git Repo Path: ' + repo_path)

Git Repo Path: repos/open-event-frontend


In [9]:
file_sha = io.open("SHAs/" + project_name + "_commits_sha.txt", 'a+')
file_sha.seek(0)

if not file_sha.read(1):
    for item in commit_list:
        file_sha.writelines(item + "\n")
    print_green("Git commit hash tags written to " + project_name + "_commits_sha file successfully.\n")
else:
    print_green(project_name + ".txt File found and opened.\n")

Git commit hash tags written to open-event-frontend_commits_sha file successfully.



Let's run analysis for every commit in the commit list.

### Trigger SonarQube Analysis

#### Date Utlities

In [10]:
def set_system_date(setdate):
    print_blue("Date changed to: %s"%setdate)
    os.system('sudo date -s "%s"' %setdate) 

def write_date(writedate):
    with io.open(project_name + "-date.txt", 'w') as datefile:
        datefile.writelines(str(writedate))

In [11]:
time_start = datetime.datetime.now()
real_date = time_start

time_count_second_prev_commits = 0
time_count_date_curr_commit_start = datetime.datetime.now()
time_count_date_curr_commit_end = datetime.datetime.now()

def count_time(isstart):
    global time_count_second_prev_commits
    global time_count_date_curr_commit_start
    global time_count_date_curr_commit_end

    if isstart == 1: 
        time_count_date_curr_commit_start = datetime.datetime.now()
    elif isstart == 0: 
        time_count_date_curr_commit_end = datetime.datetime.now()
        time_diff_seconds = (time_count_date_curr_commit_end - time_count_date_curr_commit_start).seconds
        time_count_second_prev_commits = time_count_second_prev_commits + time_diff_seconds
    else:
        print_red("Wrong value passed to isstart of count_time(isstart)!")

count_time(1)
print("Beginning date-time: " + str(real_date))

def print_time_estimation():
    global time_count_second_prev_commits
    hours, remainder = divmod(time_count_second_prev_commits, 3600)
    minutes, seconds = divmod(remainder, 60)
    print_blue('Elapsed Time for analysis: %d Hours : %d Minutes : %d Seconds'%(hours, minutes, seconds))

    if count > startat and count < stopat:
        required_time = time_count_second_prev_commits/(count-startat)
        required_time = required_time * (stopat-count)
        hours, remainder = divmod(required_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        print_blue('Estimated Time to finish:  %d Hours : %d Minutes : %d Seconds' % (hours, minutes, seconds))


def get_commit_date():
    if commit_date_mode == 1: 
        c_date = git.execute(["git", "log", "-1", "--date=local", "--pretty=format:%cd"])
        c_date = datetime.datetime.strptime(c_date, '%a %b %d %H:%M:%S %Y')
        return c_date
    elif commit_date_mode == 2:
        if count == 1:
            currentdate = initdate
            write_date(currentdate)
        else:
            with io.open(project_name + "-date.txt", 'r') as datefile:
                udate = datefile.readline()
                currentdate = datetime.datetime.strptime(udate, '%Y-%m-%d %H:%M:%S')
            currentdate = currentdate + datetime.timedelta(days=1)
            write_date(currentdate)
        return currentdate
    else:
        print_red("Wrong value for commit_date_mode!")

Beginning date-time: 2021-03-21 23:38:40.201178


In [ ]:
count = 1
count_time(0)

flag = 1
perform_sonar_analysis = 1
commit_date_mode = 1
update_date = 1
waiting_factor = 1

os.chdir(repo_path)

file_sha.seek(0)

while flag == 1:
    commit_hash = file_sha.readline()
    commit_hash = commit_hash.replace('\r', '').replace('\n', '')
    if not commit_hash:
        flag = 0
        print_green("Reached to the latest commit.")
    else:
        if count < startat:
            print_red("Skipped commit %d for analysis." % count)

        elif startat <= count <= stopat:
            count_time(1)
            
            git.execute(["git", "clean", "-df"])
            print_green("Git commit cleaned: untraced sonar property file")

            git.execute(["git", "checkout", commit_hash])
            time.sleep(0.2)
            print_blue("Git repository %s moved to commit: %d, hash: %s" % (project_name, count, commit_hash))

            count_time(0)

            print_blue('Waiting for background task to finish.')
            if time_count_second_prev_commits != 0 and (count-startat) != 0:
                print_blue('Time for last analysis: ' + str(time_count_second_prev_commits/(count-startat)) + ' seconds')
                waiting_time = (time_count_second_prev_commits/(count-startat))/waiting_factor
                if waiting_time > 3:
                    print_blue('Waiting for '+ str(waiting_time) + ' seconds')
                    time.sleep(waiting_time)
                else:
                    print_blue('Waiting for minimum 3 seconds.')
                    time.sleep(3)
            else:
                print_blue('time_count_second_prev_commits: ' + str(time_count_second_prev_commits) + ' seconds')
                time.sleep(3)

            commit_date = get_commit_date()
            set_system_date(commit_date)
            count_time(1) # start recording time

            if perform_sonar_analysis == 1:
                print_green("Starting analysing the project with sonar-scanner ...")
                subprocess.call("sonar-scanner -Dsonar.projectKey=open-event-frontend -Dsonar.projectVersion=%d -Dsonar.sources=. -Dsonar.host.url=http://34.125.191.212 -Dsonar.login=caa8c1d2448d5ea545d1df25095351c5290f0c69"%(count), shell=True)
                print_green("Completed: sonar-scanner Check.")
            else:
                print_red('Skipped Sonar-Scanner Analysis: Set perform_sonar_analysis = 1 to activate')

        elif count > stopat:
            print_green("Checked from Commit %d - %d" % (startat, stopat))
            flag = 0
            break

        count_time(0)
        print_time_estimation()
        count = count + 1

    print('-----------------------------------------------------------------------------------')

if update_date == 1:
    file_sha.close()
    estimated_date = real_date + datetime.timedelta(0, time_count_second_prev_commits)
    time.sleep(5)
    set_system_date(estimated_date)

    os.system('sudo ntpdate -s time.nist.gov')
    os.system('timedatectl set-ntp 1')
    print("Auto-update of system time enabled.")
    
else:
    print_red("Date update is disabled!")
print_red('=============================== ANALYSIS COMPLETED ================================')

Git commit cleaned: untraced sonar property file
Git repository open-event-frontend moved to commit: 1, hash: 25b1a767afe7d52d95b7757f902883d9bf5adfd1
Waiting for background task to finish.
time_count_second_prev_commits: 1 seconds
Date changed to: 2020-01-02 04:20:25
Starting analysing the project with sonar-scanner ...
Completed: sonar-scanner Check.
Elapsed Time for analysis: 0 Hours : 1 Minutes : 0 Seconds
-----------------------------------------------------------------------------------
Git commit cleaned: untraced sonar property file
Git repository open-event-frontend moved to commit: 2, hash: e8cb96bd6ef66978b6a03a2737312ff5d3637a26
Waiting for background task to finish.
Time for last analysis: 60.0 seconds
Waiting for 60.0 seconds
